# Introduction to STAN

2017-07-22; WiMLDS STAN Workshop @ Viacom (1515 Broadway)

[Documentation](http://mc-stan.org/users/documentation/index.html)

[relevant data](http://mc-stan.org/workshops/wimlds/)

I started this tutorial running python 2.7 on Anaconda2 on a 64-bit Windows 10 machine. I had some trouble getting started. The current documentation of pystan (v2.16) specifies that on Windows machines, python 3.6 is required. To install python 3.6 (or whatever the latest version of 3 is) through conda,

In [ ]:
conda create --name py36 python=3

Then, if I ever want to use pystan, make sure to switch to python 3.6 using the command prompt (on Windows):

In [ ]:
activate py36

Ideally, switching to this environment and opening a Jupyter notebook should allow you to run notebooks in python 3.6. I found that the conda install of python 3.6 doesnt include the notebook controller, which needs to be installed:

In [ ]:
conda install nb_conda

[another bug](https://blogs.msdn.microsoft.com/pythonengineering/2016/04/11/unable-to-find-vcvarsall-bat/) for this i installed the Visual Plus C++ Build Tools 2015 jk didnt work. installed 2017 free version with python development kit for python 2 and 3. also buildtools 2017

still didnt work...looks like an [open issue on mc-stan](https://github.com/stan-dev/pystan/issues/364) [another ref](http://discourse.mc-stan.org/t/cannot-compile-a-model-in-pystan-but-can-in-rstan/1114/19)  [another](https://groups.google.com/forum/#!topic/stan-users/-gHr4f72o7Q) ok did that it has similar error, but instead of status 2 with cl.exe its [status 1 error with gcc.exe](https://github.com/facebookincubator/prophet/issues/22) [fprophet has similar problem](https://github.com/facebookincubator/prophet/issues/22) try that next

Ok! Now I can get started!

In [1]:
import os
os.getcwd()
os.chdir('C:\\Users\\Dat Tien Hoang\\Downloads\\wimlds\\wimlds')
os.getcwd()

'C:\\Users\\Dat Tien Hoang\\Downloads\\wimlds\\wimlds'

In [2]:
import pystan
import stan_utility
import matplotlib
import matplotlib.pyplot as plot

# Example 1:

In [3]:
##################################################
##### Simulate data and write to file
##################################################

model = stan_utility.compile_model('C:\\Users\\Dat Tien Hoang\\Downloads\\wimlds\\wimlds\\1\\gen_data_ascii.stan')

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_1b6224d2c596fdbb3a194016f3a971e0 NOW.


CompileError: command 'C:\\Program Files (x86)\\Microsoft Visual Studio 14.0\\VC\\BIN\\x86_amd64\\cl.exe' failed with exit status 2

In [ ]:
fit = model.sampling(seed=194838, algorithm='Fixed_param', iter=1, chains=1)

data = dict(N = 25, M = 3,
            X=fit.extract()['X'][0,:,:], y = fit.extract()['y'][0,:])

pystan.stan_rdump(data, 'lin_regr.data.R')

In [ ]:
##################################################
##### Fit model and check diagnostics
##################################################

# Read in data from Rdump file
data = pystan.read_rdump('lin_regr.data.R')

# Fit posterior with Stan
model = stan_utility.compile_model('lin_regr.stan')
fit = model.sampling(data=data, seed=194838)

# Check sampler diagnostics
print(fit)

sampler_params = fit.get_sampler_params(inc_warmup=False)
stan_utility.check_div(sampler_params)
stan_utility.check_treedepth(sampler_params)
stan_utility.check_energy(sampler_params)

# Check visual diagnostics
fit.plot()
plot.show()

##################################################
##### Visualize posterior
##################################################

light="#DCBCBC"
light_highlight="#C79999"
mid="#B97C7C"
mid_highlight="#A25050"
dark="#8F2727"
dark_highlight="#7C0000"

# Plot parameter posteriors
params = fit.extract()

f, axarr = plot.subplots(2, 3)
for a in axarr[0,:]:
    a.xaxis.set_ticks_position('bottom')
    a.yaxis.set_ticks_position('none')
for a in axarr[1,:]:
    a.xaxis.set_ticks_position('bottom')
    a.yaxis.set_ticks_position('none')

axarr[0, 0].set_title("beta_1")
axarr[0, 0].hist(params['beta'][:,0], bins = 25, color = dark, ec = dark_highlight)
axarr[0, 0].axvline(x=5, linewidth=2, color=light)

axarr[0, 1].set_title("beta_2")
axarr[0, 1].hist(params['beta'][:,1], bins = 25, color = dark, ec = dark_highlight)
axarr[0, 1].axvline(x=-3, linewidth=2, color=light)

axarr[0, 2].set_title("beta_3")
axarr[0, 2].hist(params['beta'][:,2], bins = 25, color = dark, ec = dark_highlight)
axarr[0, 2].axvline(x=2, linewidth=2, color=light)

axarr[1, 0].set_title("alpha")
axarr[1, 0].hist(params['alpha'], bins = 25, color = dark, ec = dark_highlight)
axarr[1, 0].axvline(x=10, linewidth=2, color=light)

axarr[1, 1].set_title("sigma")
axarr[1, 1].hist(params['sigma'], bins = 25, color = dark, ec = dark_highlight)
axarr[1, 1].axvline(x=1, linewidth=2, color=light)

plot.show()

# Perform a posterior predictive check by plotting
# posterior predictive distributions against data
f, axarr = plot.subplots(2, 2)
for a in axarr[0,:]:
    a.xaxis.set_ticks_position('bottom')
    a.yaxis.set_ticks_position('none')
for a in axarr[1,:]:
    a.xaxis.set_ticks_position('bottom')
    a.yaxis.set_ticks_position('none')

axarr[0, 0].set_title("y_1")
axarr[0, 0].hist(params['y_ppc'][:,0], bins = 25, color = dark, ec = dark_highlight)
axarr[0, 0].axvline(x=data['y'][0], linewidth=2, color=light)

axarr[0, 1].set_title("y_5")
axarr[0, 1].hist(params['y_ppc'][:,4], bins = 25, color = dark, ec = dark_highlight)
axarr[0, 1].axvline(x=data['y'][4], linewidth=2, color=light)

axarr[1, 0].set_title("y_10")
axarr[1, 0].hist(params['y_ppc'][:,9], bins = 25, color = dark, ec = dark_highlight)
axarr[1, 0].axvline(x=data['y'][9], linewidth=2, color=light)

axarr[1, 1].set_title("y_15")
axarr[1, 1].hist(params['y_ppc'][:,14], bins = 25, color = dark, ec = dark_highlight)
axarr[1, 1].axvline(x=data['y'][14], linewidth=2, color=light)

plot.show()

# Example 1: 